In [ ]:
%pylab inline

In [ ]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

Nous avons un problème de lecture de notre dataset qui a l'air d'être trop volumineux. Nous allons d'abord regarder les contraintes que nous avons (machine, dataset) puis résoudre ce problème.

Première astuce : voici un article que j'ai écrit il y a deux ans, qui vous montrent quelques astuces pour _scaler_ Python sur de grands datasets https://blog.octo.com/machine-learning-7-astuces-pour-scaler-python-sur-de-grands-datasets/.
Au programme, suppression de variables, _downcasting_ de variables, etc.

Commençons par connaître un peu mieux les contraintes que nous avons, ici celles de notre machine dans le cloud.

# 1) Machine

Voici quelques commandes utiles. Le '!' veut dire qu'on exécute une commande bash, comme on ferait sur un terminal (sauf que là on le fait dans un notebook).

Commande pour voir la RAM : 

In [ ]:
!cat /proc/meminfo | grep Mem

65808084 ~65 go de RAM.

Voici la commande `top` pour voir les processus en cours. A décommenter si vous voulez l'utiliser. Attention il faut bien l'arrêter car sinon ça tourne indéfiniment (Kernel > Interrupt)

In [ ]:
#!top

Il existe aussi `htop` comme commande un peu plus visuelle. Vous pouvez le faire dans votre terminal sur MAC après l'avoir installée au préalable ;-).

_Fair enough_, on a une machine a priori puissante, regardons maintenant ce qu'on a comme dataset.

Si l'on va dans l'explorateur voir ce que notre fichier a dans le ventre, visiblement il fait environ 9,07 gb sur disque ; c'est donc assez gros mais pas insurmontable, sachant quand même qu'en le lisant il prendra plus souvent d'espace en mémoire. On peut souvent avoir un facteur 3 ou 4 entre le disque et la mémoire, selon le type de variables stockées dans ce fichier. Votre pire ennemi ce sont les strings, ça prend une mémoire importante. Mon intuition c'est qu'en lisant ce fichier on fait péter la RAM.

# 2) Dataset

Testons d'abord avec un échantillon, ici 3000 lignes, pour voir un peu ce qu'on a :

In [ ]:
mydataset = dataiku.Dataset("RESA_MODEL_FR_UK_GE_prepared")
df = mydataset.get_dataframe(sampling='head', limit=3000)

In [ ]:
df.head()

In [ ]:
df.info()

.info() est une commande absolument indispensable, car elle permet de connaître le type des colonnes utilisées et l'espace mémoire priis. Notamment, je vous l'ai dit, votre pire ennemi en terme d'espace mémoire ce sont les strings. Or, quand Pandas ne sait pas inférer (=deviner) le type d'une variable, il va la mettre dans une catégorie fourre-tout qui s'appelle object, et qui prend énormément d'espace mémoire.  
C'est parfois utile d'avoir des object, justement pour stocker de vraies chaines de caractères. En revanche, quand c'est pour stocker des chiffres, c'est de l'espace mémoire perdu pour rien.  
De même, float64 et int64 veulent dire qu'on va respectivement utiliser 64 bits pour stocker chaque nombre en float ou en int. C'est rarement utile d'avoir une si grande précision ; (plus de détails ici https://docs.scipy.org/doc/numpy-1.13.0/user/basics.types.html) souvent, on peut utiliser 8 bits pour coder nos chiffres de 0 à 255 (2^8), ce qui permet d'économiser énormément d'espace mémoire. Référez vous également à mon article cité plus haut si vous voulez de plus amples détails :-).

Donc la solution ici sera de lire notre fichier en lui donnant des types beaucoup plus adéquats que ce que Pandas devine par lui-même ! 

D'après la doc de Dataiku https://doc.dataiku.com/dss/latest/python-api/datasets-data.html on peut soit préciser les types de chaque colonne dans l'interface du dataset (onglet explore), soit laisser Pandas deviner. En ayant testé la première option, ça n'a pas l'air de fonctionner correctement, par exemple si on a des "Y" et des "N" la conversion en booléen est oubliée à la lecture.  
Une autre option que j'imagine c'est lire les données par chunks (= par morceaux) et inférer les types via Pandas.  

Oublions le dataset d'avant, en fait on va partir de RESA_MODEL_FR_UK_GE_prepared qui a déjà les noms de colonnes et quelques types précisés.

Je vous propose qu'on divise le dataset en plusieurs colonnes (3 parties de 20 colonnes par ex), puis on va optimiser chaque partie et enfin fusionner le tout. [Voir le schéma en annexe]   
De cette façon, on va pouvoir faire tenir tout ça dans notre RAM.

Voici ici la liste de toutes les colonnes que nous avons - à noter que j'ai enlevé les noms des colonnes pour des raisons de confidentialité, remplacez simplement par vos noms de colonnes à chaque fois que vous croisez `...`.

In [ ]:
all_columns = [...]

Pour tout ce qui suit, dans un premier temps on fait nos traitements sur 3000 lignes, ce sera plus rapide. Une fois qu'on est satisfaits de notre code, il suffit simplement de reéxécuter le tout en commentant les paramètres `sampling=head`et `limit=3000` dans `get_dataframe()`.

## Partie 1

In [ ]:
columns_part_1 = [...]

In [ ]:
df_part_1 = mydataset.get_dataframe(sampling='head', 
                                    columns=columns_part_1,
                                    limit=3000,
                                    )

In [ ]:
df_part_1.head(3)

In [ ]:
df_part_1.info()

### Suppression de variables

In [ ]:
columns_to_drop_part_1 = [...]
df_part_1 = df_part_1.drop(columns_to_drop_part_1, axis=1)

### Categories

In [ ]:
df_part_1[...] = df_part_1[...].astype('category') 

### Mapping

In [ ]:
resastatus_dic = {'A': 1, 'M': 0}
yesno_dic = {'Y': 1, 'N': 0}
yesno_columns = [...]

In [ ]:
df_part_1[...] = df_part_1[...].map(resastatus_dic)

In [ ]:
for yesno_column in yesno_columns:
    df_part_1[yesno_column] = df_part_1[yesno_column].map(yesno_dic)

### Downcasting

In [ ]:
int_columns = df_part_1.select_dtypes(include=['int']).columns.tolist()
float_columns = df_part_1.select_dtypes(include=['float']).columns.tolist()
df_part_1[int_columns] = df_part_1[int_columns].apply(pd.to_numeric, downcast='integer')
df_part_1[float_columns] = df_part_1[float_columns].apply(pd.to_numeric, downcast='float') 

### Vérification

In [ ]:
df_part_1.info()

## Partie 2

In [ ]:
columns_part_2 = [...]

In [ ]:
df_part_2 = mydataset.get_dataframe(sampling='head', 
                                    columns=columns_part_2,
                                    limit=3000,
                                    )

In [ ]:
df_part_2.head(3)

In [ ]:
df_part_2.info()

### Suppression de variables

In [ ]:
columns_to_drop_part_2 = [...]
df_part_2 = df_part_2.drop(columns_to_drop_part_2, axis=1)

### Categories

In [ ]:
df_part_2[...] = df_part_2[...].astype('category') 
df_part_2[...] = df_part_2[...].astype('category') 

### Mapping

In [ ]:
yes_no_YES_NO_dic = {'Yes': 1, 'YES':1, 'No': 0, 'NO': 0}

In [ ]:
df_part_2[...] = df_part_2[...].map(yes_no_YES_NO_dic)

### Downcasting

In [ ]:
int_columns = df_part_2.select_dtypes(include=['int']).columns.tolist()
float_columns = df_part_2.select_dtypes(include=['float']).columns.tolist()
df_part_2[int_columns] = df_part_2[int_columns].apply(pd.to_numeric, downcast='integer')
df_part_2[float_columns] = df_part_2[float_columns].apply(pd.to_numeric, downcast='float') 

### Vérification

In [ ]:
df_part_2.info()

## Partie 3

In [ ]:
columns_part_3 = [...]

In [ ]:
df_part_3 = mydataset.get_dataframe(sampling='head', 
                                    columns=columns_part_3,
                                    limit=3000,
                                    )

In [ ]:
df_part_3.head(3)

In [ ]:
df_part_3.info()

### Suppression de variables

In [ ]:
columns_to_drop_part_3 = [...]
df_part_3 = df_part_3.drop(columns_to_drop_part_3, axis=1)

### Downcasting

In [ ]:
int_columns = df_part_3.select_dtypes(include=['int']).columns.tolist()
float_columns = df_part_3.select_dtypes(include=['float']).columns.tolist()
df_part_3[int_columns] = df_part_3[int_columns].apply(pd.to_numeric, downcast='integer')
df_part_3[float_columns] = df_part_3[float_columns].apply(pd.to_numeric, downcast='float') 

### Vérification

In [ ]:
df_part_3.info()

## Fusion des dataframes

Maintenant que nous avons traité nos 3 parties, nous allons simplement refusionner le tout.

In [ ]:
df_final = pd.concat([df_part_1, df_part_2, df_part_3], axis=1)

## Sauvegarde

Et, finalement, on sauvegarde notre dataset propre. On peut aussi stocker le mapping des types utilisés, car visiblement quand on relit le dataset dans un autre notebook il faut remapper les types (par exemple, on perd les `category` qui redeviennent des `object`.

In [ ]:
df_final.info()

In [ ]:
dict(zip(list(df_final),[df_final[x].dtype.name for x in df_final]))

Avant de sauvegarder ce dataframe, on a créé un dataset au préalable dans l'interface de Dataiku.

In [ ]:
test_downcasting = dataiku.Dataset("test_downcasting")
test_downcasting.write_with_schema(df_final)